# Jupyter notebook to read the twitter data and generate network graph

## Generate followers graph

In [1]:
bjpfollowers = []
congressfollowers = []

bjpfile = open('data/bjpfollowers.txt','r')
for follower in bjpfile:
    bjpfollowers.append(follower.rstrip())
bjpfile.close()

congressfile = open('data/congressfollowers.txt','r')
for follower in congressfile:
    congressfollowers.append(follower.rstrip())
congressfile.close()

In [3]:
print(bjpfollowers[:100])
print(congressfollowers[:100])

['1159444593349259265', '1159608407801778176', '1023507413373263872', '3190206662', '1159603916926881792', '1159606199274229760', '2991146953', '874127383430934528', '1159606381428670464', '1156247658157051904', '318544244', '1159606055992623104', '1154718401991860224', '1159606199588835329', '1159386571889659910', '1004747881612562432', '1150986234178576384', '914773783491125248', '1159600932092403713', '1136092106160664576', '807511970169507841', '148698420', '1159603389174468608', '75500104', '1159602399851368448', '1159425508624482306', '858760544', '1159387197390196742', '1159295596018176000', '1159186656248786944', '1390397191', '1159592640100179968', '1024192586851000320', '1159595392540917760', '1159597866639212544', '1159598846692872194', '1157736222778900480', '1159601663252017152', '2637231171', '1159152085318811649', '744351779832291330', '1159137251881701376', '1159599140877193217', '1159024420498706433', '1098255900660453376', '1159596355410677761', '1159596679840043008',

In [5]:
import networkx as nx

G = nx.DiGraph()
attrs = {}
G.add_node('BJP')
G.add_node('Congress')
for node in bjpfollowers:
    G.add_node(node)
    G.add_edge(node,'BJP')
    attrs[node] = {'support': 1}
    if node in congressfollowers:
        G.add_edge(node,'Congress')
        attrs[node] = {'support': 2}

for node in congressfollowers:
    if node in bjpfollowers:
        continue
    G.add_node(node)
    G.add_edge(node,'Congress')
    attrs[node] = {'support': 3}
nx.set_node_attributes(G,attrs)
nx.write_gexf(G,"graphs/BJPCongressFollowers.gexf")

## Generate retweet network graph

In [59]:
tripleTalaq_file = open('data/tweets_TripleTalaq.txt','r')
unnao_file = open('data/tweets_UnnaoKiBeti.txt','r')

### Data Preprocessing

In [60]:
tripleTalaq_tweets = []
unnao_tweets = []

for line in tripleTalaq_file:
    tripleTalaq_tweets.append(line)
tripleTalaq_file.close()

for line in unnao_file:
    unnao_tweets.append(line)
unnao_file.close()

In [61]:
import json

i=0
for tweet in tripleTalaq_tweets:
    tweet_obj = json.loads(tweet)
    if 'retweeted_status' not in tweet_obj:
        del tripleTalaq_tweets[i]
    else:
        i+=1

i=0
for tweet in unnao_tweets:
    tweet_obj = json.loads(tweet)
    if 'retweeted_status' not in tweet_obj:
        del unnao_tweets[i]
    else:
        i+=1

In [62]:
print(tripleTalaq_tweets[:3])
print(unnao_tweets[:3])

['{"created_at": "Tue Aug 06 22:11:46 +0000 2019", "id": 1158863258838683648, "id_str": "1158863258838683648", "text": "@AzizKavish Now this issue has been converted into #TripleTalaq \\ud83d\\ude02", "truncated": false, "entities": {"hashtags": [{"text": "TripleTalaq", "indices": [51, 63]}], "symbols": [], "user_mentions": [{"screen_name": "AzizKavish", "name": "Kavish", "id": 309152096, "id_str": "309152096", "indices": [0, 11]}], "urls": []}, "metadata": {"iso_language_code": "en", "result_type": "recent"}, "source": "<a href=\\"http://twitter.com/download/android\\" rel=\\"nofollow\\">Twitter for Android</a>", "in_reply_to_status_id": 1158433081352548352, "in_reply_to_status_id_str": "1158433081352548352", "in_reply_to_user_id": 309152096, "in_reply_to_user_id_str": "309152096", "in_reply_to_screen_name": "AzizKavish", "user": {"id": 792050421212872705, "id_str": "792050421212872705", "name": "Nadeem Irfan", "screen_name": "NadeemIrfan18", "location": "ULTA PRADESH \\ud83d\\ude05\\

### Modeling and Graph generation

In [67]:
import networkx as nx

G = nx.Graph()

tweets_file = open('data/tweets_TripleTalaq.txt','r')

for line in tweets_file:
    tweet_obj = json.loads(line)
    if 'retweeted_status' in tweet_obj:
        #print(tweet_obj['retweeted_status']['user']['screen_name'] + ' -> ' + tweet_obj['user']['screen_name'])
        A = tweet_obj['retweeted_status']['user']['screen_name']
        B = tweet_obj['user']['screen_name']
        G.add_node(A)
        G.add_node(B)
        G.add_edge(A,B)

tweets_file.close()

tweets_file = open('data/tweets_UnnaoKiBeti.txt','r')

for line in tweets_file:
    tweet_obj = json.loads(line)
    if 'retweeted_status' in tweet_obj:
        #print(tweet_obj['retweeted_status']['user']['screen_name'] + ' -> ' + tweet_obj['user']['screen_name'])
        A = tweet_obj['retweeted_status']['user']['screen_name']
        B = tweet_obj['user']['screen_name']
        G.add_node(A)
        G.add_node(B)
        G.add_edge(A,B)

tweets_file.close()

nx.write_gexf(G, "graphs/tweet_graph.gexf")